In [2]:
import requests


In [25]:
# Retrieve and save all information about a sample of completed mythic runs.
base_uri = "https://raider.io/api/v1/mythic-plus/runs"
season = "season-df-2"
region = "world"
dungeon = "all"
affixes = "all"

raw_rankings = []
for i in range(1):
    params = {
        "season": season,
        "region": region,
        "dungeon": dungeon,
        "affixes": affixes,
        "page": i
    }
    data = requests.get(base_uri, params=params).json()
    raw_rankings.extend(data["rankings"])

# Flatten Data
def flatten_data(ranking):
    flattened_ranking = {
        "score": ranking["score"],
        "dungeon_name": ranking["run"]["dungeon"]["name"],
        "dungeon_num_bosses": ranking["run"]["dungeon"]["num_bosses"],
        "mythic_level": ranking["run"]["mythic_level"],
        "clear_time_ms": ranking["run"]["clear_time_ms"],
        "keystone_time_ms": ranking["run"]["keystone_time_ms"],
        "weekly_modifiers": [i["name"] for i in ranking["run"]["weekly_modifiers"]]
    }
    
    dps_count = 1
    for i in ranking["run"]["roster"]:
        role = i["role"]
        charclass = i["character"]["class"]["name"]
        race = i["character"]["race"]["name"]
        level = i["character"]["level"]
        spec = i["character"]["spec"]["name"]
        if role == "tank":
            tank_details = {
                "tank_class": charclass,
                "tank_race": race,
                "tank_level": level,
                "tank_spec": spec
            }
            flattened_ranking.update(tank_details)
        if role == "healer":
            healer_details = {
                "healer_class": charclass,
                "healer_race": race,
                "healer_level": level,
                "healer_spec": spec
            }
            flattened_ranking.update(healer_details)
        if role == "dps":
            dps_details = {
                f"dps_{dps_count}_class": charclass,
                f"dps_{dps_count}_race": race,
                f"dps_{dps_count}_level": level,
                f"dps_{dps_count}_spec": spec
            }
            dps_count += 1
            flattened_ranking.update(dps_details)
    return flattened_ranking

rankings = [flatten_data(i) for i in raw_rankings]
rankings

[{'score': 213.1,
  'dungeon_name': 'Freehold',
  'dungeon_num_bosses': 4,
  'mythic_level': 26,
  'clear_time_ms': 1644119,
  'keystone_time_ms': 1800999,
  'weekly_modifiers': ['Tyrannical', 'Incorporeal', 'Spiteful'],
  'tank_class': 'Demon Hunter',
  'tank_race': 'Night Elf',
  'tank_level': 70,
  'tank_spec': 'Vengeance',
  'dps_1_class': 'Rogue',
  'dps_1_race': 'Dwarf',
  'dps_1_level': 70,
  'dps_1_spec': 'Subtlety',
  'dps_2_class': 'Priest',
  'dps_2_race': 'Night Elf',
  'dps_2_level': 70,
  'dps_2_spec': 'Shadow',
  'dps_3_class': 'Shaman',
  'dps_3_race': 'Dwarf',
  'dps_3_level': 70,
  'dps_3_spec': 'Enhancement',
  'healer_class': 'Paladin',
  'healer_race': 'Dwarf',
  'healer_level': 70,
  'healer_spec': 'Holy'},
 {'score': 212.6,
  'dungeon_name': 'The Underrot',
  'dungeon_num_bosses': 4,
  'mythic_level': 26,
  'clear_time_ms': 1710426,
  'keystone_time_ms': 1800999,
  'weekly_modifiers': ['Fortified', 'Afflicted', 'Raging'],
  'dps_1_class': 'Mage',
  'dps_1_race': 